# Credit Risk Resampling Techniques

In [208]:
import warnings
warnings.filterwarnings('ignore')

In [209]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV and Perform Basic Data Cleaning

In [210]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


# Split the Data into Training and Testing

In [211]:
# Create our features
X = df["loan_status"]

# Create our target
y = df.drop(columns="loan_status")


In [212]:
X.describe()

count        77536
unique           2
top       low_risk
freq         75036
Name: loan_status, dtype: object

In [213]:
# Check the balance of our target values
df["loan_status"].value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [214]:
# Create X_train, X_test, y_train, y_test

from sklearn.datasets import make_blobs

X, y = make_blobs(centers=2, random_state=1, cluster_std=3)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)

X_train.shape


(75, 2)

# Simple Logistic Regression

In [215]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [216]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9583333333333333

In [217]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[11,  1],
       [ 0, 13]], dtype=int64)

In [218]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.92      1.00      0.96      0.96      0.91        12
          1       0.93      1.00      0.92      0.96      0.96      0.92        13

avg / total       0.96      0.96      0.96      0.96      0.96      0.92        25



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests


### Naive Random Oversampling

In [238]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler

model_ros = RandomOverSampler(random_state=1)

X_res, y_res = model_ros.fit_resample(X_train, 
                                      y_train)

Counter(y_res)

Counter({1: 38, 0: 38})

In [239]:
# Train the Logistic Regressionmodel using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_res, y_res)

LogisticRegression(random_state=1)

In [240]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9583333333333333

In [241]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[11,  1],
       [ 0, 13]], dtype=int64)

In [242]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.92      1.00      0.96      0.96      0.91        12
          1       0.93      1.00      0.92      0.96      0.96      0.92        13

avg / total       0.96      0.96      0.96      0.96      0.96      0.92        25



### SMOTE Oversampling

In [243]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE

X_res, y_res = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(
    X_train, y_train
)
from collections import Counter

Counter(y_res)

Counter({1: 38, 0: 38})

In [244]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_res, y_res)

LogisticRegression(random_state=1)

In [245]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9583333333333333

In [246]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[11,  1],
       [ 0, 13]], dtype=int64)

In [247]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.92      1.00      0.96      0.96      0.91        12
          1       0.93      1.00      0.92      0.96      0.96      0.92        13

avg / total       0.96      0.96      0.96      0.96      0.96      0.92        25



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [248]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import RandomUnderSampler

rosunder = RandomUnderSampler(random_state=1)

X_res, y_res = rosunder.fit_resample(X_train, 
                                     y_train)

from collections import Counter

Counter(y_res)

Counter({0: 37, 1: 37})

In [249]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_res, y_res)

LogisticRegression(random_state=1)

In [250]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.9583333333333333

In [251]:
# Display the confusion matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[11,  1],
       [ 0, 13]], dtype=int64)

In [252]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.92      1.00      0.96      0.96      0.91        12
          1       0.93      1.00      0.92      0.96      0.96      0.92        13

avg / total       0.96      0.96      0.96      0.96      0.96      0.92        25



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [253]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN

sm = SMOTEENN(random_state=1)
X_res, y_res = sm.fit_resample(X_train,
                               y_train)

Counter(y_res)

Counter({0: 37, 1: 34})

In [254]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_res, y_res)

LogisticRegression(random_state=1)

In [255]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.9583333333333333

In [256]:
# Display the confusion matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[12,  0],
       [ 0, 13]], dtype=int64)

In [257]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      1.00      1.00      1.00      1.00      1.00        12
          1       1.00      1.00      1.00      1.00      1.00      1.00        13

avg / total       1.00      1.00      1.00      1.00      1.00      1.00        25

